In [2]:
import string
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords
def netoyage(corpus_ensemble_documents):
    #transformer tous en miniscules
    for i in range(len(corpus_ensemble_documents)):
        corpus_ensemble_documents[i]=corpus_ensemble_documents[i].lower()
    for i in range(len(corpus_ensemble_documents)):
        #eliminer les ponctuations
        for c in string.punctuation:
                x=corpus_ensemble_documents[i].replace(c,' ')
                corpus_ensemble_documents[i]=x
    stopwords_francais=stopwords.words('french')
    for i in range(len(corpus_ensemble_documents)):
        L=corpus_ensemble_documents[i].split()
        for mot in L:
            if mot in stopwords_francais:
                while mot in L:
                    L.remove(mot)
        corpus_ensemble_documents[i]=" ".join(L)
    return(corpus_ensemble_documents)

In [4]:
texte=["La vie est douce","La vie est tranquille, est belle, est douce","Bonjour je mappelle nisrine bajja"]
netoyage(texte)

['vie douce', 'vie tranquille belle douce', 'bonjour mappelle nisrine bajja']

In [5]:
from numpy import *
#la frequence du terme==>l'importance de ce terme term frequency
def TF(terme,corpus,numero_document):
    x=corpus[numero_document].count(terme)
    y=len(corpus[numero_document].split())
    return x/y

In [6]:
def IDF(terme,corpus,numero_document):
    D=len(corpus)
    d_t=0 #les nombres de document qui contient au moins le terme
    for document in corpus:
        if terme in document:
            d_t+=1
    TF_val=TF(terme,corpus,numero_document)
    return TF_val*log(1+(D/d_t))

def cle_correspondante_a_valeur(valeur,dictionnaire):
    for cle in dictionnaire.keys():
        if dictionnaire[cle]==valeur:
            return cle

In [7]:
import numpy
def matrice_sparse(dictionnaire,corpus_ensemble_document):
    M=numpy.zeros((len(corpus_ensemble_document),len(dictionnaire.values())))#lignes : longueur du corpus
    #colonne : les valeurs contenat dans le document
    for i in range(len(corpus_ensemble_document)):
        for j in dictionnaire.values():
            x=cle_correspondante_a_valeur(j,dictionnaire)
            M[i,j]=IDF(x,corpus_ensemble_document,i)
    return M

In [8]:
def affiche(M):
    (n,p)=M.shape
    for i in range(n):
        for j in range(p):
            M[i,j]=round(M[i,j],2)
    print(M)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
texte=["La vie est douce","La vie est tranquille, est belle, est douce","Le corona-virus est méchant"]
texte=netoyage(texte)
print(texte)
vect=TfidfVectorizer()
T=vect.fit_transform(texte)
dictionnaire_des_mots=vect.vocabulary_
print("dictionnaire_des_mots :",dictionnaire_des_mots)
liste_des_mots=list(dictionnaire_des_mots.keys())
print("list_des_mots :",liste_des_mots)
Matrice_sparse_correspondante=T.toarray()
print("matrice_sparse_correspondante_methode_predefinie :/n")
print(Matrice_sparse_correspondante)
print("matrice_sparse_correspondante_notre_methode :/n")
print(matrice_sparse(dictionnaire_des_mots,texte))

['vie douce', 'vie tranquille belle douce', 'corona virus méchant']
dictionnaire_des_mots : {'vie': 5, 'douce': 2, 'tranquille': 4, 'belle': 0, 'corona': 1, 'virus': 6, 'méchant': 3}
list_des_mots : ['vie', 'douce', 'tranquille', 'belle', 'corona', 'virus', 'méchant']
matrice_sparse_correspondante_methode_predefinie :/n
[[0.         0.         0.70710678 0.         0.         0.70710678
  0.        ]
 [0.5628291  0.         0.42804604 0.         0.5628291  0.42804604
  0.        ]
 [0.         0.57735027 0.         0.57735027 0.         0.
  0.57735027]]
matrice_sparse_correspondante_notre_methode :/n
[[0.         0.         0.45814537 0.         0.         0.45814537
  0.        ]
 [0.34657359 0.         0.22907268 0.         0.34657359 0.22907268
  0.        ]
 [0.         0.46209812 0.         0.46209812 0.         0.
  0.46209812]]


In [18]:
import numpy as np
import pandas
from sklearn.model_selection import train_test_split
spams = pandas.read_table("/Users/mac/Downloads/SMSSpamCollection.txt",sep="\t",header=0)
#print(spams)
spamsTrain, spamsTest = train_test_split(spams,train_size=0.7,random_state=1)

from sklearn.feature_extraction.text import CountVectorizer
parseur=CountVectorizer()
Xtrain = parseur.fit_transform(spamsTrain['message'])
#print(Xtrain)
mdtTrain = Xtrain.toarray()
print(mdtTrain)
from sklearn.linear_model import LogisticRegression
modelFirst = LogisticRegression()

modelFirst.fit(mdtTrain,spamsTrain['classe'])
score1=modelFirst.score(mdtTrain, spamsTrain['classe'])
print("score sur data train :"+str(score1))
mdtTest = parseur.transform(spamsTest['message'])
score2=modelFirst.score(mdtTest,spamsTest['classe'])
print("score sur data test : "+str(score2))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
score sur data train :0.9976923076923077
score sur data test : 0.9814593301435407
